# Data Mining Project
---

Import βιβλιοθήκες

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
# df_raw = pd.read_csv('household_power_consumption.txt', sep=';', low_memory=False)
#
# day_data = df_raw[df_raw['Date'] == '28/4/2007']
#
# print("Sample of missing values on 28/04/2007: ")
# display(day_data.iloc[15:30])

In [25]:
df = pd.read_csv('household_power_consumption.txt', sep=';', na_values=['?'], low_memory=False)

print("\nNumber of missing values before cleaning")
print(df.isna().sum())


Number of missing values before cleaning
Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


Συμπλήρωση ελλιπών τιμών με ffill()

In [26]:
df.ffill(inplace=True)

In [27]:
print("\nNumber of missing values after cleaning")
print(df.isna().sum())


Number of missing values after cleaning
Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


Μετατροπή σε Datetime

In [28]:
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)
df.set_index('datetime', inplace=True)
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Δημιουργία του Sub_metering_4 (Λοιπές συσκευές)
# Τύπος ακριβώς όπως ορίζεται στα Notes:
# (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3)

df['Sub_metering_4'] = (df['Global_active_power'] * 1000 / 60) - df['Sub_metering_1'] - df['Sub_metering_2'] - df['Sub_metering_3']

# Καθαρισμός τυχόν αρνητικών τιμών που μπορεί να προκύψουν από μικρο-αποκλίσεις μετρήσεων
df['Sub_metering_4'] = df['Sub_metering_4'].clip(lower=0)

print("Το χαρακτηριστικό 'Sub_metering_4' δημιουργήθηκε.")

# Έλεγχος: Εμφάνιση των πρώτων γραμμών
print("Δείγμα υπολογισμού Sub_metering_4:")
display(df[['Global_active_power', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Sub_metering_4']].head())

print(df.head())

Το χαρακτηριστικό 'Sub_metering_4' δημιουργήθηκε.
Δείγμα υπολογισμού Sub_metering_4:


,Global_active_power,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4
datetime,,,,,
2006-12-16 17:24:00,4.216,0.0,1.0,17.0,52.266667
2006-12-16 17:25:00,5.360,0.0,1.0,16.0,72.333333
2006-12-16 17:26:00,5.374,0.0,2.0,17.0,70.566667
2006-12-16 17:27:00,5.388,0.0,1.0,17.0,71.800000
2006-12-16 17:28:00,3.666,0.0,1.0,17.0,43.100000


                     Global_active_power  Global_reactive_power  Voltage  \
datetime                                                                   
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:24:00              18.4             0.0             1.0   
2006-12-16 17:25:00              23.0             0.0             1.0   
2006-12-16 17:26:00              23.0             0.0             2.0   
2006-12-16 17:27:00              23.0             0.0             1.0   
2006-12-16 17:28:00          

Επαναδειγματοληψία (Resampling)

In [29]:
df_daily = df.resample('D').agg({ # D = daily
    'Global_active_power': 'sum',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'sum',
    'Sub_metering_2': 'sum',
    'Sub_metering_3': 'sum',
    'Sub_metering_4': 'sum'
})

Δημιουργία Daily_total_power

In [30]:
df_daily['Daily_total_power'] = df_daily['Global_active_power'] * (1000 / 60) #Wh

Δημιουργία Peak_hour_power

In [31]:
hourly_consumption = df['Global_active_power'].resample('h').sum() # h = hourly
peak_hour_series = hourly_consumption.resample('D').max()

Δημιουργία Nighttime_usage

In [32]:
night_mask = (df.index.hour >= 22) | (df.index.hour < 6)
df_night = df[night_mask]
night_usage_series = df_night['Global_active_power'].resample('D').sum()

Προσθήκη χαρακτηριστικών ως columns

In [33]:
df_daily['Peak_hour_power'] = peak_hour_series
df_daily['Night_usage_power'] = night_usage_series

Lag Features

In [34]:
# katanalwsh akribws prohgoumenhs meras
df_daily['Power_lag_1'] = df_daily['Daily_total_power'].shift(1)

# katanalwsh prin 2 meres
df_daily['Power_lag_2'] = df_daily['Daily_total_power'].shift(2)

# katanalwsh idias meras prin 1 bdomada
df_daily['Power_lag_7'] = df_daily['Daily_total_power'].shift(7)

#  kyliomenos m.o 3 hmerwn (tash)
df_daily['Power_rolling_mean_3'] = df_daily['Daily_total_power'].shift(1).rolling(window=3).mean()

#  kyliomenos m.o 3 hmerwn (tash)
df_daily['Power_rolling_mean_7'] = df_daily['Daily_total_power'].shift(1).rolling(window=7).mean()

# ta prwta rows exoun kenes times logw tou shift
df_daily.dropna(inplace=True)
df_daily.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Daily_total_power,Peak_hour_power,Night_usage_power,Power_lag_1,Power_lag_2,Power_lag_7,Power_rolling_mean_3,Power_rolling_mean_7
datetime,,,,,,,,,,,,,,,,
2006-12-23,4773.386,0.153799,240.136076,14.028056,2669.0,425.0,14726.0,61736.433333,79556.433333,333.748,1254.266,39022.300000,28618.533333,20152.933333,34912.211111,35128.223810
2006-12-24,2550.012,0.104097,241.687437,7.640417,1703.0,5082.0,6891.0,28824.200000,42500.200000,241.680,923.256,79556.433333,39022.300000,56507.666667,49065.755556,43614.438095
2006-12-25,2743.120,0.166861,243.399931,7.951528,6620.0,1962.0,5795.0,31341.666667,45718.666667,252.892,674.410,42500.200000,79556.433333,36730.433333,53692.977778,41613.371429
2006-12-26,3934.110,0.114654,241.625438,11.347917,1086.0,2533.0,14979.0,46970.500000,65568.500000,242.514,1001.936,45718.666667,42500.200000,27769.900000,55925.100000,42897.404762
2006-12-27,1528.760,0.124237,243.767361,4.517500,0.0,314.0,6976.0,18189.333333,25479.333333,141.186,290.858,65568.500000,45718.666667,37095.800000,51262.455556,48297.204762


Κατηγορικά χαρακτηριστικά

In [35]:
df_daily['Day_of_week'] = df_daily.index.dayofweek
df_daily['Is_workday'] = df_daily['Day_of_week'].apply(lambda x: 1 if x < 5 else 0) # Δευτέρα - Παρασκευή
df_daily['Is_weekend'] = df_daily['Day_of_week'].apply(lambda x: 1 if x >= 5 else 0) # ΣΚ
df_daily['Month'] = df_daily.index.month

In [36]:
def get_season(month):
    if month in [12,1,2]: return 1 # Winter
    elif month in [3,4,5]: return 2 # Spring
    elif month in [6,7,8]: return 3 # Summer
    else: return 4 # Fall

df_daily['Season'] = df_daily['Month'].apply(get_season)

In [47]:
# one hot encoders gia tis epoxes
season_dummies = pd.get_dummies(df_daily['Season'], prefix='Season')

# Μετονομασία των στηλών για να είναι πιο κατανοητές
season_dummies.columns = ['Is_Winter', 'Is_Spring', 'Is_Summer', 'Is_Autumn']

# 2. Ένωση των νέων στηλών με το κύριο dataset μας
df_daily = pd.concat([df_daily, season_dummies], axis=1)

# 3. Προαιρετικά: Διαγραφή της αρχικής στήλης 'Season' και 'Month'
# αφού πλέον έχουμε τις πληροφορίες σε bit μορφή
df_daily.drop(['Season'], axis=1, inplace=True)

In [48]:
# 1. Δημιουργία των dummy μεταβλητών για τις ημέρες (0=Monday, ..., 6=Sunday)
day_dummies = pd.get_dummies(df_daily['Day_of_week'], prefix='Day')

# 2. Μετονομασία των στηλών για να είναι ξεκάθαρες στην αναφορά σου
day_dummies.columns = [
    'Is_Monday', 'Is_Tuesday', 'Is_Wednesday',
    'Is_Thursday', 'Is_Friday', 'Is_Saturday', 'Is_Sunday'
]

# 3. Ένωση (Concat) με το κύριο dataset
df_daily = pd.concat([df_daily, day_dummies], axis=1)

# Προαιρετικά: Μπορείς να διαγράψεις την αρχική στήλη Day_of_week
df_daily.drop(['Day_of_week'], axis=1, inplace=True)

In [49]:
# 1. Δημιουργία των dummy μεταβλητών για τους μήνες (1=Ιανουάριος, ..., 12=Δεκέμβριος)
month_dummies = pd.get_dummies(df_daily['Month'], prefix='Month')

# 2. Μετονομασία των στηλών για να είναι ξεκάθαρες
month_dummies.columns = [
    'Is_Jan', 'Is_Feb', 'Is_Mar', 'Is_Apr', 'Is_May', 'Is_Jun',
    'Is_Jul', 'Is_Aug', 'Is_Sep', 'Is_Oct', 'Is_Nov', 'Is_Dec'
]

# 3. Ένωση με το κύριο dataset
df_daily = pd.concat([df_daily, month_dummies], axis=1)

# Προαιρετικά: Διαγραφή της αρχικής στήλης Month
df_daily.drop(['Month'], axis=1, inplace=True)

In [50]:
df_daily.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Daily_total_power,Peak_hour_power,...,Is_Mar,Is_Apr,Is_May,Is_Jun,Is_Jul,Is_Aug,Is_Sep,Is_Oct,Is_Nov,Is_Dec
datetime,,,,,,,,,,,,,,,,,,,,,
2006-12-23,4773.386,0.153799,240.136076,14.028056,2669.0,425.0,14726.0,61736.433333,79556.433333,333.748,...,False,False,False,False,False,False,False,False,False,True
2006-12-24,2550.012,0.104097,241.687437,7.640417,1703.0,5082.0,6891.0,28824.200000,42500.200000,241.680,...,False,False,False,False,False,False,False,False,False,True
2006-12-25,2743.120,0.166861,243.399931,7.951528,6620.0,1962.0,5795.0,31341.666667,45718.666667,252.892,...,False,False,False,False,False,False,False,False,False,True
2006-12-26,3934.110,0.114654,241.625438,11.347917,1086.0,2533.0,14979.0,46970.500000,65568.500000,242.514,...,False,False,False,False,False,False,False,False,False,True
2006-12-27,1528.760,0.124237,243.767361,4.517500,0.0,314.0,6976.0,18189.333333,25479.333333,141.186,...,False,False,False,False,False,False,False,False,False,True


Δημιουργία Weekend_usage

In [51]:
df_daily['Weekend_usage'] = df_daily.apply(
    lambda row: row['Daily_total_power'] if row['Is_weekend'] == 1 else 0,
    axis=1
)

In [52]:
df_daily.fillna(0, inplace=True)

In [53]:
print(df_daily.head())

            Global_active_power  Global_reactive_power     Voltage  \
datetime                                                             
2006-12-23             4773.386               0.153799  240.136076   
2006-12-24             2550.012               0.104097  241.687437   
2006-12-25             2743.120               0.166861  243.399931   
2006-12-26             3934.110               0.114654  241.625438   
2006-12-27             1528.760               0.124237  243.767361   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
datetime                                                                       
2006-12-23         14.028056          2669.0           425.0         14726.0   
2006-12-24          7.640417          1703.0          5082.0          6891.0   
2006-12-25          7.951528          6620.0          1962.0          5795.0   
2006-12-26         11.347917          1086.0          2533.0         14979.0   
2006-12-27          4.517500 

κανονικοποιηση

In [57]:
cols_to_scale = [
    'Daily_total_power', 'Global_reactive_power', 'Voltage',
    'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
    'Sub_metering_3', 'Sub_metering_4', 'Peak_hour_power',
    'Night_usage_power', 'Power_lag_1', 'Power_lag_7', 'Power_rolling_mean_3', 'Power_rolling_mean_7'
]

scaler = StandardScaler()

df_daily_scaled = df_daily.copy()
df_daily_scaled[cols_to_scale] = scaler.fit_transform(df_daily[cols_to_scale])

check_cols = ['Daily_total_power', 'Voltage', 'Sub_metering_1']
print(df_daily_scaled[check_cols].describe().loc[['mean', 'std']].round(2))

      Daily_total_power  Voltage  Sub_metering_1
mean               -0.0      0.0            -0.0
std                 1.0      1.0             1.0


In [55]:
print(df_daily_scaled[cols_to_scale].describe().round(2))

       Daily_total_power  Global_reactive_power  Voltage  Global_intensity  \
count            1435.00                1435.00  1435.00           1435.00   
mean               -0.00                  -0.00     0.00             -0.00   
std                 1.00                   1.00     1.00              1.00   
min                -2.24                  -3.51    -4.76             -2.31   
25%                -0.66                  -0.71    -0.38             -0.64   
50%                -0.04                  -0.13     0.06             -0.04   
75%                 0.55                   0.52     0.51              0.54   
max                 5.33                   4.75     3.22              5.45   

       Sub_metering_1  Sub_metering_2  Sub_metering_3  Sub_metering_4  \
count         1435.00         1435.00         1435.00         1435.00   
mean            -0.00           -0.00           -0.00            0.00   
std              1.00            1.00            1.00            1.00   
min  

In [58]:
# Υπολογίζουμε τον μέσο όρο της συνολικής κατανάλωσης
mean_power = df_daily['Daily_total_power'].mean()

# Δημιουργούμε τη μεταβλητή-στόχο (Target Variable)
# 1 = Υψηλή κατανάλωση (πάνω από τον μέσο όρο)
# 0 = Κανονική κατανάλωση (κάτω από τον μέσο όρο)
df_daily_scaled['High_Consumption'] = (df_daily['Daily_total_power'] > mean_power).astype(int)

print(f"Το threshold ορίστηκε στα {mean_power:.2f} Wh.")
print("Κατανομή κλάσεων:")
print(df_daily_scaled['High_Consumption'].value_counts())

Το threshold ορίστηκε στα 26008.44 Wh.
Κατανομή κλάσεων:
High_Consumption
0    734
1    701
Name: count, dtype: int64


Converting to CSV

In [59]:
# Ορισμός του ονόματος του αρχείου
file_name = 'household_power_final_preprocessed.csv'

# Αποθήκευση σε CSV
# index=True: Σώζουμε και την ημερομηνία (datetime index) γιατί τη χρειαζόμαστε
df_daily_scaled.to_csv(file_name, index=True)

print(f"Το αρχείο '{file_name}' δημιουργήθηκε επιτυχώς!")
print(f"Συνολικές γραμμές: {len(df_daily_scaled)}")
print(f"Συνολικά χαρακτηριστικά (columns): {len(df_daily_scaled.columns)}")

Το αρχείο 'household_power_final_preprocessed.csv' δημιουργήθηκε επιτυχώς!
Συνολικές γραμμές: 1435
Συνολικά χαρακτηριστικά (columns): 66
